In [1]:
# sandbox to test on new hampshire

In [2]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# We will work on New Hampshire
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).
The NJ data is formatted the same way, just replace 'nh' with 'nj'

### This is the current assignment of precinct to congressional districts (2 of them for NH)

In [3]:
nh_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nh.csv')
nh_current_assignment

,GEOID20,District
0,33007KILK01,2
1,33007STAR01,2
2,33007RAND01,2
3,33007BERL01,2
4,33007MILA01,2
...,...,...
321,33003CHAT01,1
322,33007BPUR01,2
323,33007SHEL01,2
324,33007BERL03,2


### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [4]:
nh_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nh.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nh_precinct_data = nh_precinct_data[keepcolumns]
nh_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,33003FREE01,1,1090,538,538,1689,1607,12,1,21,31,1
1,33003WAKE01,1,3270,1240,1986,5201,4881,65,35,50,105,7
2,33003EFFI01,1,926,391,526,1691,1578,28,12,16,44,0
3,33003BART01,1,2170,1374,773,3200,3041,38,30,25,55,3
4,33003TAMW01,1,1768,899,838,2812,2635,35,14,30,58,2
...,...,...,...,...,...,...,...,...,...,...,...,...
321,33007CHAN01,2,0,0,0,0,0,0,0,0,0,0
322,33003HALE01,1,117,31,85,132,125,1,1,1,0,0
323,33015ZZZZZZ,1,0,0,0,0,0,0,0,0,0,0
324,33007BGRT01,2,0,0,0,0,0,0,0,0,0,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [5]:
shpfile = 'Map_Data/nh_vtd_2020_bound/nh_vtd_2020_bound.shp'
dbffile = 'Map_Data/nh_vtd_2020_bound/nh_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nh_vtd_2020_bound/nh_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nh_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nh_precinct_boundaries[rec[3]]=geom

### 3. From an Existing map - Flip Step

Let's try a random-based strategy which takes a precinct on the border of the two districts and flip it to the other district.
Let's do this 10 times starting from the current official assignment map.

In [6]:
nh_flipstep_assignment = nh_current_assignment.copy()

## Let's do a simplistic approach. 
## First we randomly select 5 D1 precincts that border D2 and assign them to D2

D1_border_precincts = []

D1_list = nh_latitude_assignment[nh_latitude_assignment['District']==1]['GEOID20']
for precinct in D1_list:
    neighbors = ast.literal_eval(nh_contiguity[nh_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
    for neighbor in neighbors:
        if nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==2:
            #if one of the neighbor is in D2, then this is a border district
            D1_border_precincts.append(precinct)
            break
#Sample 5 and flip them UNLESS it breaks contiguity
flipped_precincts = np.random.choice(D1_border_precincts,5)
for flip in flipped_precincts:
    nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=2
    #check if we broke contiguity and revert the flip if we did
    if(isDistrictContiguous(1,nh_flipstep_assignment,nh_contiguity) is False or isDistrictContiguous(2,nh_flipstep_assignment,nh_contiguity) is False):
        nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=1
        print("Contiguity broken " + flip)
                                                                        

## First we randomly select 5 DIFFERENT D2 precincts that border D1 and assign them to D1

D2_border_precincts = []
D2_list = nh_latitude_assignment[nh_latitude_assignment['District']==2]['GEOID20']
for precinct in D2_list:
    neighbors = ast.literal_eval(nh_contiguity[nh_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
    for neighbor in neighbors:
        if nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==neighbor,'District'].values.tolist()[0]==1 and precinct not in flipped_precincts:
            #if one of the neighbor is in D2, then this is a border district. We do not want to re-flip a district we just flipped
            D2_border_precincts.append(precinct)
            break
#Sample 5 and flip them UNLESS it breaks contiguity
flipped_precincts = np.random.choice(D2_border_precincts,5)
for flip in flipped_precincts:
    nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=1
    #check if we broke contiguity and revert the flip if we did
    if(isDistrictContiguous(1,nh_flipstep_assignment,nh_contiguity) is False or isDistrictContiguous(2,nh_flipstep_assignment,nh_contiguity) is False):
        nh_flipstep_assignment.loc[nh_flipstep_assignment['GEOID20']==flip,'District']=2
        print("Contiguity broken " + flip)
                                                        
    


    

print(isDistrictContiguous(1,nh_flipstep_assignment,nh_contiguity))
print(isDistrictContiguous(2,nh_flipstep_assignment,nh_contiguity))

nh_flipstep_assignment.to_csv('Recitation maps/nh_flipstep_assignment.csv',index=False)


NameError: name 'nh_latitude_assignment' is not defined